<a href="https://colab.research.google.com/github/Vindhya-Singh/Master-s-Thesis/blob/master/LDA_AmazonProductData_ToolAndHomeImprovement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pandas nltk gensim pyLdavis

     |████████████████████████████████| 1.6MB 2.8MB/s 
  Created wheel for pyLdavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=f3a231a6e64ec7ae4cc256c25ecb9d4f296f0baa06de9e14be0e2352a2368f43
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLdavis


## All the Imports go here

In [ ]:
import re # We clean text using regex
import csv # To read the csv
from collections import defaultdict # For accumlating values
from nltk.corpus import stopwords # To remove stopwords
from gensim import corpora # To create corpus and dictionary for the LDA model
from gensim.models import LdaModel # To use the LDA model
import pyLDAvis.gensim # To visualise LDA model effectively
import pandas as pd

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

##Reading the Amazon Product Review file 

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving reviews_Tools_and_Home_Improvement.json.gz to reviews_Tools_and_Home_Improvement.json.gz
User uploaded file "reviews_Tools_and_Home_Improvement.json.gz" with length 381777044 bytes


In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/reviews_Tools_and_Home_Improvement.json.gz')

df.tail()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
1926042,AGRJ3SRUYQOF8,B00LLGSPQ8,Mike,"[0, 0]",This company truly has great quality of lighti...,5.0,"Great staff, quality, and company",1404691200,"07 7, 2014"
1926043,A2YGGF3929H3IC,B00LOGKKL8,Jerry,"[0, 0]","Great hinge, Strongly recommended, i'm using i...",5.0,Five Stars,1404950400,"07 10, 2014"
1926044,A3IXXERDW8EYZI,B00LPQE0ZY,darby j plath,"[0, 0]",This works great! It fixed my gaps that I cou...,5.0,Works great!,1405814400,"07 20, 2014"
1926045,ACX42MH818TLY,B00LV6T90E,"Amazon Customer ""fil""","[0, 0]",I purchased this item from Amazon and got an e...,1.0,a product in question and horrible service,1406073600,"07 23, 2014"
1926046,A22UHERHRWEB78,B00LV6T90E,Paul S.,"[3, 3]",I bought three of these from Amazon three week...,2.0,Potential Issue Reported by Amazon,1406073600,"07 23, 2014"


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('reviews_samples.csv')
!cp reviews_samples.csv drive/My\ Drive/

In [ ]:
from itertools import islice
fileContents = defaultdict(list)
with open('/content/drive/My Drive/reviews_samples.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in islice(reader, 50000): # read a row as {column1: value1, column2: value2,...}
        for (k,v) in row.items(): # go over each column name and value 
            fileContents[k].append(v) # append the value into the appropriate list

In [ ]:
reviews = fileContents['reviewText']

In [ ]:
reviews = [re.sub(r'[^\w\s]','',str(item)) for item in reviews]

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
texts = [[word for word in document.lower().split() if word not in stopwords] for document in reviews]

##LDA Implementation

In [ ]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [ ]:
dictionary = corpora.Dictionary(texts)

In [ ]:
print(dictionary)

Dictionary(29117 unique tokens: ['amazon', 'beautiful', 'beauty', 'christmas', 'gift']...)


In [ ]:
# So what does the dictionary look like : Dictionary(29117 unique tokens: ['amazon', 'beautiful', 'beauty', 'christmas', 'gift']...)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
NUM_TOPICS = 5 # This is an assumption. Needs to be tuned. 
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)#This might take some time.

In [ ]:
topics = ldamodel.show_topics()
for topic in topics:
    print(topic)

(0, '0.023*"tool" + 0.018*"use" + 0.014*"dremel" + 0.013*"drill" + 0.012*"great" + 0.010*"well" + 0.010*"bit" + 0.009*"work" + 0.009*"used" + 0.008*"good"')
(1, '0.009*"one" + 0.008*"use" + 0.007*"dust" + 0.006*"tape" + 0.006*"like" + 0.006*"ive" + 0.006*"used" + 0.006*"get" + 0.005*"sander" + 0.005*"dont"')
(2, '0.020*"light" + 0.011*"great" + 0.009*"use" + 0.009*"works" + 0.008*"easy" + 0.007*"well" + 0.007*"one" + 0.006*"like" + 0.006*"good" + 0.006*"flashlight"')
(3, '0.020*"one" + 0.013*"battery" + 0.010*"product" + 0.009*"would" + 0.009*"new" + 0.009*"years" + 0.009*"good" + 0.008*"price" + 0.008*"time" + 0.008*"bought"')
(4, '0.040*"saw" + 0.021*"blade" + 0.016*"table" + 0.013*"cut" + 0.008*"cuts" + 0.008*"blades" + 0.007*"cutting" + 0.007*"fence" + 0.006*"base" + 0.006*"dewalt"')


In [ ]:
word_dict = {};
for i in range(NUM_TOPICS):
    words = ldamodel.show_topic(i, topn = 20)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,tool,one,light,one,saw
1,use,use,great,battery,blade
2,dremel,dust,use,product,table
3,drill,tape,works,would,cut
4,great,like,easy,new,cuts
5,well,ive,well,years,blades
6,bit,used,one,good,cutting
7,work,get,like,price,fence
8,used,sander,good,time,base
9,good,dont,flashlight,bought,dewalt


In [ ]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
